In [49]:
import os
import pandas as pd
import numpy as np
import math
import re
import nltk
from sklearn.datasets import load_files
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Data load

In [64]:
data_path = "../data/news_script_integrated/"
news_list = sorted(os.listdir(data_path))
news_list

['20201.txt',
 '202010.txt',
 '202011.txt',
 '20202.txt',
 '20203.txt',
 '20204.txt',
 '20205.txt',
 '20206.txt',
 '20207.txt',
 '20208.txt',
 '20209.txt']

In [66]:
# data = pd.DataFrame()
stemmer = WordNetLemmatizer()
data = {}

In [67]:
def text_preprocess(text):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', text)
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    return document    

## Text preprocess

In [108]:
for i in range(len(news_list)):
    f = open(data_path + news_list[i], 'r', encoding='UTF-8')
    lines = f.readlines()
    text = []
    for line in lines:
        tmp = text_preprocess(line).split()
        if len(tmp) > 0:
            text.append(text_preprocess(line).split())
    break

## Pos tagging

In [74]:
from gensim.models import Word2Vec, FastText

In [111]:
time Cbow_model = Word2Vec(text, size=100, window=3, min_count=1,  workers=1, sg=0, iter=10)

Wall time: 948 ms


In [112]:
Cbow_model.wv.vocab.keys()

dict_keys(['in', 'country', 'with', 'one', 'of', 'the', 'world', 'lowest', 'birth', 'rate', 'would', 'assume', 'news', 'beloved', 'celebrity', 'announcing', 'plan', 'for', 'marriage', 'and', 'an', 'imminent', 'child', 'be', 'welcomed', 'open', 'arm', 'but', 'not', 'korea', 'k', 'pop', 'idol', 'chen', 'member', 'most', 'successful', 'group', 'exo', 'announced', 'handwritten', 'letter', 'posted', 'on', 'social', 'medium', 'that', 'he', 'wa', 'doing', 'exactly', 'this', 'yet', 'much', 'response', 'from', 'fan', 'both', 'toxic', 'unpleasant', 'many', 'called', 'him', 'to', 'leave', 'immediately', 'spoke', 'betrayal', 'they', 'felt', 'such', 'devil', 'comment', 'ang', 'pul', 'a', 'are', 'known', 'korean', 'should', 'taken', 'lightly', 'recent', 'suicide', 'star', 'sulli', 'goo', 'hara', 'have', 'been', 'linked', 'negative', 'treatment', 'received', 'online', 'twitter', 'filled', 'hashtags', 'retweeted', 'ten', 'thousand', 'wildly', 'criticizing', 'his', 'perceived', 'crime', 'inconsiderate'

In [114]:
Cbow_model.wv.most_similar("song", topn=10)

[('album', 0.9998980760574341),
 ('single', 0.9998947381973267),
 ('from', 0.9998942017555237),
 ('for', 0.999893307685852),
 ('released', 0.9998928308486938),
 ('with', 0.9998924136161804),
 ('while', 0.9998920559883118),
 ('seoul', 0.9998915195465088),
 ('including', 0.9998871684074402),
 ('track', 0.9998854398727417)]